In [1]:
import os
import pandas as pd
import numpy as np
from PIL import Image
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
import torch
import torch.nn as nn
import torch.optim as optim
import random

# 경로 설정
train_dir = 'cv_challenge/label_change/train'
test_dir = 'cv_challenge/test'
train_csv_path = 'cv_challenge/label_change/train.csv'

# 시드값 설정
seed = 42
torch.manual_seed(seed)
np.random.seed(seed)
random.seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False


# 데이터셋 클래스 정의
class ImageDataset(Dataset):
    def __init__(self, image_dir, csv_file=None, transform=None):
        self.image_dir = image_dir
        self.csv_file = csv_file
        self.transform = transform
        if csv_file is not None:
            self.df = pd.read_csv(csv_file)
        else:
            self.df = None
        self.image_names = os.listdir(image_dir)
    
    def __len__(self):
        return len(self.image_names)
    
    def __getitem__(self, idx):
        img_name = self.image_names[idx]
        img_path = os.path.join(self.image_dir, img_name)
        image = Image.open(img_path).convert('RGB')
        if self.transform:
            image = self.transform(image)
        if self.df is not None:
            label = self.df[self.df['ID'] == img_name]['target'].values[0]
            return image, label
        else:
            return image

# 이미지 전처리 정의
transform = transforms.Compose([
    transforms.Resize((512, 512)),
    transforms.ToTensor()
])

# 데이터셋 및 데이터로더 정의
train_dataset = ImageDataset(train_dir, train_csv_path, transform)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

test_dataset = ImageDataset(test_dir, transform=transform)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

In [2]:


class Autoencoder(nn.Module):
    def __init__(self):
        super(Autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2, padding=0),
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        )
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(64, 32, kernel_size=2, stride=2),
            nn.ReLU(),
            nn.ConvTranspose2d(32, 32, kernel_size=2, stride=2),
            nn.ReLU(),
            nn.Conv2d(32, 3, kernel_size=3, stride=1, padding=1),
            nn.Sigmoid()
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

autoencoder = Autoencoder().cuda()
criterion = nn.MSELoss()
optimizer = optim.Adam(autoencoder.parameters(), lr=0.001)

In [4]:
# 학습 함수 정의
def train_autoencoder(model, criterion, optimizer, data_loader, num_epochs=50):
    model.train()
    for epoch in range(num_epochs):
        running_loss = 0.0
        for images, _ in data_loader:
            images = images.cuda()
            outputs = model(images)
            loss = criterion(outputs, images)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(data_loader)}')

# 오토인코더 학습
train_autoencoder(autoencoder, criterion, optimizer, train_loader)

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\master\\cv_challenge\\label_change\\train\\desktop.ini'